model traning using Knowledge Graph Embeddings

first step

In [2]:
%tensorflow_version

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


checking tensorflow version

In [1]:
%%capture
# Install AmpliGraph library
! pip install ampligraph

# Required to visualize embeddings with tensorboard projector, comment out if not required!
! pip install --user tensorboard

# Required to plot text on embedding clusters, comment out if not required!
! pip install --user git+https://github.com/Phlya/adjustText

install AmpliGraph and other dependencies  to use in neural link

In [3]:
from ampligraph.datasets import load_fb15k_237, load_wn18rr, load_yago3_10

Loading a Knowledge Graph dataset called freebase 237

In [4]:
import pandas as pd

URL = 'https://ampgraphenc.s3-eu-west-1.amazonaws.com/datasets/freebase-237-merged-and-remapped.csv'
dataset = pd.read_csv(URL, header=None)
dataset.columns = ['subject', 'predicate', 'object']
dataset.head(5)

,subject,predicate,object
0,"queens college, city university of new york",/education/educational_institution/students_gr...,carol leifer
1,digital equipment corporation,/business/business_operation/industry,computer hardware
2,/m/0drtv8,/award/award_ceremony/awards_presented./award/...,laurence mark
3,the departed,/award/award_winning_work/awards_won./award/aw...,leonardo dicaprio
4,marilyn manson,/people/person/profession,actor


In [5]:
print('Total triples in the KG:', dataset.shape)

Total triples in the KG: (310079, 3)


total kG

In [6]:
from ampligraph.evaluation import train_test_split_no_unseen
# get the validation set of size 500
test_train, X_valid = train_test_split_no_unseen(dataset.values, 500, seed=0)

# get the test set of size 1000 from the remaining triples
X_train, X_test = train_test_split_no_unseen(test_train, 1000, seed=0)

print('Total triples:', dataset.shape)
print('Size of train:', X_train.shape)
print('Size of valid:', X_valid.shape)
print('Size of test:', X_test.shape)

Total triples: (310079, 3)
Size of train: (308579, 3)
Size of valid: (500, 3)
Size of test: (1000, 3)


 Create training, validation and test splits

In [ ]:
from ampligraph.latent_features import TransE
model = TransE(k=150,                                                             # embedding size
               epochs=100,                                                        # Num of epochs
               batches_count= 10,                                                 # Number of batches
               eta=1,                                                             # number of corruptions to generate during training
               loss='pairwise', loss_params={'margin': 1},                        # loss type and it's hyperparameters
               initializer='xavier', initializer_params={'uniform': False},       # initializer type and it's hyperparameters
               regularizer='LP', regularizer_params= {'lambda': 0.001, 'p': 3},   # regularizer along with its hyperparameters
               optimizer= 'adam', optimizer_params= {'lr': 0.001},                # optimizer to use along with its hyperparameters
               seed= 0, verbose=True)

model.fit(X_train)

from ampligraph.utils import save_model, restore_model
save_model(model, 'TransE-small.pkl')

Model Training

In [ ]:
import numpy as np
list_of_actors = ['salma hayek', 'carrie fisher', 'natalie portman',  'kristen bell',
                  'mark hamill', 'neil patrick harris', 'harrison ford' ]

# stack it horizontally to create s, p, o
hypothesis = np.column_stack([list_of_actors,
                              ['/film/actor/film./film/performance/film'] * len(list_of_actors),
                              ['star wars'] * len(list_of_actors),
                             ])

# score the hypothesis
triple_scores = model.predict(hypothesis)

# append the scores column
scored_hypothesis = np.column_stack([hypothesis, triple_scores])
# sort by score in descending order
scored_hypothesis = scored_hypothesis[np.argsort(scored_hypothesis[:, 3])]
scored_hypothesis

list of hypothesis and score them

for each test set triple <s, p, o>:
           a. Compute the score of the test triple (hypothesis)
               hypothesis_score = score(<s, p, o>)

           b. Generate the subject corruptions
                   sub_corr = <?, p, o>
           c. Compute the score of the subject corruptions
                   sub_corr_score = score(sub_corr)
           d. Find the position of hypothesis_score in sub_corr_score to get the sub_rank

           e. Generate the object corruption
                   obj_corr = <s, p, ?>
           f. Compute the score of the object corruptions
                   obj_corr_score = score(obj_corr)
           g. Find the position of hypothesis_score in obj_corr_score to get the obj_rank

           h. Return rank = [sub_rank, obj_rank]

In [ ]:
from ampligraph.utils import create_tensorboard_visualizations

model = restore_model('TransE.pkl')

create_tensorboard_visualizations(model, 'embeddings_transe')

Visualizing embeddings using Tensorboard